# Introduction to Graph based RAG using LangChain and Neo4j

Prereqs: 
* Python is installed in your machine

* Run to create a separate virtual env for the project, disconnect VPN first:
`conda create -n <YOUR_DESIRED_PYTHON_ENV_NAME>`

* Run to switch to the created virtual env:
`conda activate <YOUR_DESIRED_PYTHON_ENV_NAME>`

* Run to install python packages in requirements.txt while avoiding SSL cert verify failed issue when installing
`pip install -r requirements.txt --trusted-host pypi.org --trusted-host files.pythonhosted.org`

* Install Neo4j Desktop and create a new database: [Neo4j Desktop download](https://neo4j.com/download/) (local instance). [How to create a new database](https://neo4j.com/docs/desktop-manual/current/operations/create-dbms/)

* Install the [`APOC` plugin](https://neo4j.com/labs/apoc/4.1/installation/#neo4j-desktop)

* Environment variables for `NEO4J_URI` (usually `bolt://localhost:7687` ), `NEO4J_PASSWORD`, `NEO4J_USERNAME`, and `OPENAI_API_KEY` should be set. The Neo4j URI, username, and password can be obtained from the Neo4j Desktop. The OpenAI API key can be obtained from your OpenAI account

## Load libraries

In [3]:
# Set up the credentials
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_APIKEY"
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "<YOUR_NEO4J_DB_PASSWORD"


In [43]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    ConfigurableField, RunnableParallel, RunnablePassthrough
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from importlib import reload
import utils

import warnings
warnings.filterwarnings('ignore')
#reload(utils)

### Connect to the Neo4j database 
Connect to the graph database using the [LangChain Neo4jGraph connector](https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html).

In [69]:
# Create a Neo4jGraph object to interact with the graph database
graph = Neo4jGraph(database="neo4j")

### Data Ingestion
Load 1MDB scandal Wikipedia page using the [LangChain Wikipedia loader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html)

In [70]:

# Read the wikipedia article using WikipediaLoader 
raw_documents = WikipediaLoader(query="1Malaysia Development Berhad scandal", load_max_docs=1, doc_content_chars_max=47568).load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
# Split the documents into chunks based on the chunking strategy
documents = text_splitter.split_documents(raw_documents)

In [71]:
raw_documents

[Document(metadata={'title': '1Malaysia Development Berhad scandal', 'summary': 'The 1Malaysia Development Berhad scandal, often referred to as the 1MDB scandal or just 1MDB, is an ongoing corruption, bribery and money laundering conspiracy in which the Malaysian sovereign wealth fund 1Malaysia Development Berhad (1MDB) was systematically embezzled, with assets diverted globally by the perpetrators of the scheme. Although it began in 2009 in Malaysia, the scandal\'s global scope implicated institutions and individuals in politics, banking, and entertainment, and led to criminal investigations in a number of nations. The 1MDB scandal has been described as "one of the world\'s greatest financial scandals" and declared by the United States Department of Justice as the "largest kleptocracy case to date" in 2016.\nA 2015 document leak reported in The Edge, Sarawak Report, and The Wall Street Journal showed that Malaysia\'s then-Prime Minister Najib Razak had channeled over RM 2.67 billion (

In [72]:
documents

[Document(metadata={'title': '1Malaysia Development Berhad scandal', 'summary': 'The 1Malaysia Development Berhad scandal, often referred to as the 1MDB scandal or just 1MDB, is an ongoing corruption, bribery and money laundering conspiracy in which the Malaysian sovereign wealth fund 1Malaysia Development Berhad (1MDB) was systematically embezzled, with assets diverted globally by the perpetrators of the scheme. Although it began in 2009 in Malaysia, the scandal\'s global scope implicated institutions and individuals in politics, banking, and entertainment, and led to criminal investigations in a number of nations. The 1MDB scandal has been described as "one of the world\'s greatest financial scandals" and declared by the United States Department of Justice as the "largest kleptocracy case to date" in 2016.\nA 2015 document leak reported in The Edge, Sarawak Report, and The Wall Street Journal showed that Malaysia\'s then-Prime Minister Najib Razak had channeled over RM 2.67 billion (

### Construct a graph based on the retrieved documents
The [LLMGraphTransformer](https://python.langchain.com/v0.2/api_reference/experimental/graph_transformers/langchain_experimental.graph_transformers.llm.LLMGraphTransformer.html) returns graph documents, which can be imported to the Neo4j graph database via the [add_graph_documents](https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html#langchain_community.graphs.neo4j_graph.Neo4jGraph.add_graph_documents) method. The `baseEntityLabel` parameter assigns an additional __Entity__ label to each node, enhancing indexing and query performance. The `include_source` parameter links nodes to their originating documents, facilitating data traceability and context understanding. Define the LLM for knowledge graph generation chain usage.

In [47]:
# Create the LLM using gpt-4o
llm=ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Convert the documents to graph documents
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# Add the documents to the graph
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [73]:
graph_documents

[GraphDocument(nodes=[Node(id='1Mdb', type='Organization', properties={}), Node(id='1Malaysia Development Berhad Scandal', type='Event', properties={}), Node(id='Najib Razak', type='Person', properties={}), Node(id='Jho Low', type='Person', properties={}), Node(id='United States Department Of Justice', type='Organization', properties={}), Node(id='The Edge', type='Organization', properties={}), Node(id='Sarawak Report', type='Organization', properties={}), Node(id='The Wall Street Journal', type='Organization', properties={}), Node(id='Equanimity', type='Object', properties={}), Node(id='Red Granite Pictures', type='Organization', properties={}), Node(id='The Wolf Of Wall Street', type='Creative_work', properties={}), Node(id='Rosmah Mansor', type='Person', properties={}), Node(id='Riza Aziz', type='Person', properties={}), Node(id='Muhyiddin Yassin', type='Person', properties={}), Node(id='Abdul Gani Patail', type='Person', properties={}), Node(id='Mahathir Mohamad', type='Person', pr

### Inspect the generated graph with yfiles visualization.

In [74]:
# directly show the graph resulting from the given Cypher query
default_cypher = '''
MATCH (s)-[r]->(t)
WHERE NOT type(r) = 'MENTIONS'
RETURN s, r, t
LIMIT 50
'''

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = utils.neo4j_driver
    # create a graph widget to display the graph
    session = driver.session(database="neo4j")
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

Let's take a deeper look into the graphs in Neo4j

In [ ]:
# person with most relationships
'''
MATCH (n:Person)-[r]-()
RETURN n, COUNT(r) AS total_count
ORDER BY total_count DESC
LIMIT 1
'''

# all relationships of that person
'''
MATCH (n:Person {id: 'PERSON_ID'})-[r]-(t)
WHERE NOT type(r) = 'MENTIONS'
RETURN n, r, t
'''

# Traditional vs Hybrid Retrieval (GraphRAG)
Enhancing RAG with Graph-based retrieval

## Unstructured data retriever
Semantic search through vector embeddings and vector search. The `Neo4jVector.from_existing_graph` method adds vector retrieval to documents. It configures vector search indexes for a retrieval approach.

In [50]:
# Create the vector index from the graph 
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="vector",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


In [138]:
# Cipher query to check embeddings generated
'''
MATCH (n:Document)
RETURN n.text, n.embedding
LIMIT 5
'''

'\nMATCH (n:Document)\nRETURN n.text, n.embedding\nLIMIT 5\n'

In [75]:
vector_index.similarity_search("1MDB")

[Document(metadata={'summary': 'The 1Malaysia Development Berhad scandal, often referred to as the 1MDB scandal or just 1MDB, is an ongoing corruption, bribery and money laundering conspiracy in which the Malaysian sovereign wealth fund 1Malaysia Development Berhad (1MDB) was systematically embezzled, with assets diverted globally by the perpetrators of the scheme. Although it began in 2009 in Malaysia, the scandal\'s global scope implicated institutions and individuals in politics, banking, and entertainment, and led to criminal investigations in a number of nations. The 1MDB scandal has been described as "one of the world\'s greatest financial scandals" and declared by the United States Department of Justice as the "largest kleptocracy case to date" in 2016.\nA 2015 document leak reported in The Edge, Sarawak Report, and The Wall Street Journal showed that Malaysia\'s then-Prime Minister Najib Razak had channeled over RM 2.67 billion (about US$700 million) into his personal bank acco

## Graph retriever
The graph retriever starts by identifying relevant entities in the input. For simplicity, we instruct the LLM to identify people, organizations, and locations. To achieve this, we will use LCEL with the `with_structured_output` method to achieve this.

LCEL stands for "Language Chain Extraction Language". In this case, the prompt is chained to the LLM, and the LLM is expected to produce a structured output adhering to the Entities class

In [18]:
# Just creating index for node entities
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

# Extract entities from text
entity_chain = prompt | llm.with_structured_output(Entities)

### Test Graph retriever to detect entities in the question

In [76]:

entity_chain.invoke({"question": "What scandal is Najib Razak known for in 1MDB?"})

Entities(names=['Najib Razak', '1MDB'])

### Define query generation function
Now that we can detect entities in the question, let's use a full-text index to map them to the knowledge graph. 
First, we define a full-text index and a function that will generate full-text queries that allow a bit of misspelling.

In [21]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

The `structured_retriever` function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes. 

In [77]:
print(structured_retriever("Who is Najib Razak and what is 1MDB?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

Najib Razak - MEMBER -> Umno
Najib Razak - COLLABORATED -> Jho Low
Najib Razak - SPOUSE -> Rosmah Mansor
Najib Razak - INVOLVED_IN -> 1Mdb Scandal
Najib Razak - ASSOCIATED_WITH -> Jho Low
Najib Razak - ESTABLISHED -> 1Malaysia Development Berhad
Najib Razak - MARRIED_TO -> Rosmah Mansor
Najib Razak - GUILTY_OF_CHARGES -> Src International
Najib Razak - PRIME_MINISTER -> Malaysia
Najib Razak - CHAIRMAN -> 1Mdb
Najib Razak - JOINT_VENTURE -> Petrosaudi
Najib Razak - LOAN_APPROVAL -> 1Mdb
Najib Razak - ARREST_WARRANT -> Jho Low
Najib Razak - RECEIVED_FUNDS -> Falcon Bank
Najib Razak - CHARGED -> Jho Low
Najib Razak - GUILTY -> Src International
Najib Razak - MET -> Jho Low
Najib Razak - MET -> Turki Bin Abdullah Al Saud
Najib Razak - MET -> Tarek Obaid
Najib Tun Razak - RELATED_TO -> Ipic
Najib Tun Razak - RELATED_TO -> Najib'S Stepson
Najib Tun Razak - MISAPPROPRIATED_FUNDS -> Src International
Najib Tun Razak - ASKED_FOR_HELP -> Mohammed Bin Zayed
Najib Tun Razak - FOUND_GUILTY_ON -> 28

### Final retriever
Bringing it all together, combine the unstructured and graph retriever to create the final context that will be passed to an LLM.

In [28]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(final_data)
    return final_data

### Defining the RAG chain

Now that we have successfully implemented the RAG retrieval component above, let's add query rewriting that allows conversational follow up questions.

Chat history and user question is sent to LLM to be "summarised" as a condensed question 

In [29]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

### Defining the prompt
Next, we introduce a prompt that leverages the context provided by the integrated hybrid (Graph + Vector Search) retriever to produce the response, completing the implementation of the RAG chain.

If there is any chat history, it is summarised first using _search_query, and then piped to retriever
* Graph query: cipher query to retrieve relationships of the entity from question
* Semantic search: vector search the embeddings to retrieve context 

In [32]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise. If you don't know the answer, you can say "I don't know." 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            # if there is any chat history, it is summarised first using _search_query, and then piped to retriever
            # retriever uses both graph query to retrieve relationships of the entity from question
            # and vector search the embeddings to retrieve context 
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

### Test the hybrid RAG implementation

In [82]:
chain.invoke({"question": "How is Jho Low related to Najib?"})

Search query: Who should be responsible for 1MDB scandal?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Structured data:
1Mdb - REPORTED_TO -> Companies Commission Of Malaysia
1Mdb - RELATED_TO -> Src International
1Mdb - ADVISED_BY -> Ernst & Young
1Mdb - FUNDED -> Red Granite Pictures
1Mdb - CAUSE_CRISIS_OF_CONFIDENCE -> Malaysia
1Mdb - DEBT_INVOLVEMENT -> Goldman Sachs
1Mdb - JOINT_VENTURE -> Petrosaudi International
1Mdb - RAISED_FUNDS -> Rm_3.5_Billion
1Mdb - AUDITED_BY -> Kpmg
1Mdb - VALUED_BY -> Deloitte
1Mdb - CONTRACT_AWARD_DATE -> April_2013
1Mdb - AWARDED_CONTRACT -> Perbadanan_Perwira_Harta_Malaysia
1Mdb - AGREEMENT_DATE -> 31_December_2015
1Mdb - AUDITOR -> Deloitte_Malaysia
1Mdb - DEBTS_ACCUMULATED_BY -> Early_2015
1Mdb - PAID -> Aabar Investments Pjs
1Mdb - PURCHASED_ASSETS -> Genting Group
1Mdb - TRANSFERRED_FUNDS_TO -> Najib Razak
1Mdb - TRANSFERRED_FUNDS_TO -> Ambank
1Mdb - TRANSFERRED_FUNDS_TO -> Affin Bank
1Mdb - TRANSFERRED_FUNDS_TO -> British Virgin Islands-Registered Company
1Mdb - JOINT_VENTURE_AGREEMENT -> Dusable Capital Management Llc
1Mdb - CREATED_FUND -> Yur

'The 1MDB scandal involved multiple individuals and entities, but key figures include Najib Razak, the former Prime Minister of Malaysia, and Jho Low, who is alleged to be the mastermind behind the embezzlement of funds. Additionally, institutions like Goldman Sachs were implicated in the scandal. Responsibility is shared among these individuals and entities involved in the misappropriation and mismanagement of 1MDB funds.'

In [79]:

chain.invoke(
    {
        "question": "Was he also arrested?",
        "chat_history": [("How is Jho Low related to Najib?", "Jho Low is associated with Najib Razak through collaboration and ties. Jho Low collaborated with Najib in the 1MDB scheme and has ties to him.")],
    }
)

Search query: Was Jho Low also arrested?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Structured data:
Jho Low - SUPPORTED -> 1Malaysia Development Berhad
Jho Low - COLLABORATED -> Petrosaudi
Jho Low - LAWSUIT -> Doj
Jho Low - ASSOCIATED_WITH -> Fugees
Jho Low - SETTLEMENT -> Doj
Jho Low - CHARGED_WITH -> Money Laundering
Jho Low - FUNDED -> 2012 Presidential Election
Jho Low - PURCHASED_WITH_STOLEN_FUNDS -> Equanimity
Jho Low - PASSPORT_REVOKED_BY -> Cypriot Cabinet
Jho Low - INVESTIGATION -> 1Mdb
Jho Low - MISAPPROPRIATED_FUNDS -> 1Mdb
Jho Low - CHARGED_WITH_MONEY_LAUNDERING -> 1Mdb
Jho Low - MISAPPROPRIATION -> 1Mdb
Jho Low - CHARGED -> Doj
Jho Low - MISAPPROPRIATED_FUNDS_FROM -> 1Mdb
Jho Low - FORFEITED_ASSETS_TO -> Doj
Low Taek Jho - CONSULTED -> 1Mdb
Low Taek Jho - SIPHONED_FUNDS -> Good Star Ltd
Low Taek Jho - HAS_TIES -> Najib Razak
Low Taek Jho - SIPHONED_FUNDS_THROUGH -> Good Star Ltd
Najib Razak - COLLABORATED -> Jho Low
Najib Razak - ASSOCIATED_WITH -> Jho Low
Us Department Of Justice - INVESTIGATED -> Jho Low
1Malaysia Development Berhad Scandal - INVOLVES 

'Jho Low was not arrested. The Home Ministry and Interpol were unsuccessful in locating him, and he remains a fugitive.'

## Baseline RAG

The retriever is written without graph query. Just vector search

In [57]:
def traditional_retriever(question: str):
    print(f"Search query: {question}")
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(final_data)
    return final_data

In [58]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise. If you don't know the answer, you can say "I don't know." 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

traditional_chain = (
    RunnableParallel(
        {
            "context": _search_query | traditional_retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

## Let's compare...

In [83]:
# Question asked to RAG
traditional_chain.invoke({"question": "How is Mahathir connected to Najib through the 1MDB scandal? Please explain as indepth as possible in 500 words."})

Search query: How is Mahathir connected to Najib through the 1MDB scandal? Please explain as indepth as possible in 500 words.
Unstructured data:

text:  together political figures and other organisations in calling for Najib Razak's resignation or removal. The Malaysian Conference of Rulers called for prompt investigation of the scandal, saying that it was causing a crisis of confidence in Malaysia.
Outside of Malaysia, investigations into financial and criminal activity relating to 1MDB opened in at least six countries. According to its publicly filed accounts, 1MDB had nearly RM 42 billion (US$11.73 billion) in debt by 2015. Some of this debt resulted from a $3 billion state-guaranteed 2013 bond issue led by the American investment bank Goldman Sachs, which had been reported to have received fees of up to $300 million for the deal, although the bank disputes this figure. Nevertheless, Goldman Sachs was charged under the Foreign Corrupt Practices Act and agreed to pay over $2.9 billi

"Mahathir Mohamad and Najib Razak are connected through the 1MDB scandal primarily due to their political roles and the actions Mahathir took in response to the scandal. Mahathir Mohamad, a former Prime Minister of Malaysia, became a vocal critic of Najib Razak, who was the Prime Minister during the height of the 1MDB scandal. The scandal involved the embezzlement of billions of dollars from the Malaysian sovereign wealth fund, 1Malaysia Development Berhad (1MDB), and implicated Najib in significant financial misconduct.\n\nMahathir, who had previously served as Malaysia's Prime Minister from 1981 to 2003, returned to the political scene as a critic of Najib. He was particularly concerned about the allegations of corruption and the impact of the scandal on Malaysia's international reputation and economic stability. Mahathir's criticism of Najib was not only based on the financial mismanagement but also on Najib's attempts to suppress investigations and silence critics, including the di

In [84]:
# Question asked to GraphRAG
chain.invoke({"question": "How is Mahathir connected to Najib through the 1MDB scandal? Please explain as indepth as possible in 500 words."})

Search query: How is Mahathir connected to Najib through the 1MDB scandal? Please explain as indepth as possible in 500 words.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

Structured data:
Mahathir - FILED_LAWSUIT_AGAINST -> Najib Razak
Mahathir Mohamad - REOPEN_INVESTIGATION -> 1Mdb
Mahathir Mohamad - WITHDREW_SUPPORT -> Najib
Mahathir Mohamad - QUIT -> Umno
Mahathir Mohamad - CHAIR -> Malaysian Citizens' Declaration
1Malaysia Development Berhad Scandal - INVOLVES -> Mahathir Mohamad
Pakatan Harapan - LED_BY -> Mahathir MohamadNooryana Najwa Najib - MET_WITH -> Tim Leissner
Nooryana Najwa Najib - MET_WITH -> Roger Ng
Nooryana Najwa Najib - APPLIED_FOR_JOB_AT -> Goldman Sachs Group Inc
Nooryana Najwa Najib - APPLIED_FOR_JOB -> Goldman Sachs Group Inc
Najib - OUSTED_IN -> 2018_General_Election
Najib - FUNDS_DIVERTED_FROM -> 1Mdb
Najib - OUSTER_RELATED -> Audit_Report
Najib - SENTENCED_TO -> 12 Years' Jail
Najib - FINED -> Rm 210 Million
Najib - SPENT_MONEY_IN -> United States
Najib - SPENT_MONEY_IN -> Singapore
Najib - SPENT_MONEY_IN -> Italy
Macc - INVESTIGATED -> Najib
Tony Pua - QUESTIONED -> Najib
Macc - STATEMENT_ABOUT -> Najib
Wan_Adnan_Wan_Mamat - 

"Mahathir Mohamad's connection to Najib Razak through the 1MDB scandal is multifaceted, involving political, legal, and investigative dimensions. Mahathir, a former Prime Minister of Malaysia, played a significant role in the political landscape surrounding the 1MDB scandal, which was a massive corruption and money laundering conspiracy involving the Malaysian sovereign wealth fund, 1Malaysia Development Berhad (1MDB).\n\n1. **Political Opposition and Criticism**: Mahathir Mohamad was a vocal critic of Najib Razak, who was the Prime Minister of Malaysia during the height of the 1MDB scandal. Mahathir, once an ally of Najib, became one of his most prominent adversaries. He withdrew his support for Najib and quit the United Malays National Organization (UMNO), the ruling party led by Najib, in protest against the handling of the 1MDB scandal. Mahathir chaired the Malaysian Citizens' Declaration, which brought together political figures and organizations calling for Najib's resignation or